### Test the processImage pipeline on a single image

In [1]:
TEST_IMAGE_PATH = "test_images/solidYellowCurve.jpg"
import os
imgs = os.listdir("test_images/")
imgs = ["test_images/" + x for x in  imgs ]
IMAGES = [TEST_IMAGE_PATH, "all"]
IMAGES = IMAGES + imgs

hyperParamsDefault = {
    "blurKernalSize": (5,1,29,2),  # (value, min, max, step)
    "cannyHT": (300,0,500,10),
    "cannyLT": (150,0,500,10),
    "images": IMAGES,
    "outputStage": ["final", "final", "gray", "blur", "canny"]
}


['test_images/solidYellowCurve.jpg', 'all', 'test_images/solidWhiteCurve.jpg', 'test_images/solidWhiteRight.jpg', 'test_images/solidYellowCurve.jpg', 'test_images/solidYellowCurve2.jpg', 'test_images/solidYellowLeft.jpg', 'test_images/whiteCarLaneSwitch.jpg']


In [2]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import helpers
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

%load_ext autoreload
%autoreload 2

from helpers import *
from lanedetector import detectLanes
from io import BytesIO
from base64 import b64encode
from PIL import Image
from IPython.core.display import display, HTML


In [3]:

def parseParams(hyParams):
    outP = {}
    for k in hyParams:
        try:
            outP[k] = hyParams[k][0]
        except:
            outP[k] = hyParams[k]
    return outP

def parseParamsInteract(hyParams):
    outP = {}
    for k in hyParams:
        try:
            outP[k] = (hyParams[k][1:])
        except:
            outP[k] = hyParams[k]
    return outP
        
hyperParams = parseParams(hyperParamsDefault)
hyperParamsInteract = parseParamsInteract(hyperParamsDefault)

In [4]:
# %load lanedetector.py
from helpers import *

def detectLanes(img, params):

    blkrSize = params["blurKernalSize"]
    cht = params["cannyHT"]
    clt = params["cannyLT"]
    stage = params["outputStage"]

    gray = grayscale(img)
    blur = gaussian_blur(gray, kernel_size=blkrSize)
       
    ht = cht
    lt = clt
    canny_edges = canny(blur, low_threshold=lt, high_threshold=ht)

    outpDict = {
            "final": canny_edges,
            "blur": blur,
            "gray": gray,
            "canny": canny_edges,
    }

    return outpDict[stage]


In [5]:
# InLine image display helpers

def getPngBytes(img):
    pil_im = Image.fromarray(img)
    b = BytesIO()  
    pil_im.save(b, format='png')
    return b.getvalue()
    
def displaySideBySide(im1, im2):
    
    b1 = getPngBytes(im1)
    b2 = getPngBytes(im2)
    b641 = b64encode(b1).decode('utf-8')
    b642 = b64encode(b2).decode('utf-8')
    HTMLDATA = "<table><tr><td><img src='data:image/png;base64,{0}'/></td><td><img src='data:image/png;base64,{1}'/></td></tr></table>".format(b641, b642)
    display(HTML(HTMLDATA))

In [6]:
# Interactive parameter tuning

PLAYED_PARAMS = {}

def assignDefaults(wids, defVal):
    valDict = {}
    for wid in wids.children:
        try:
            valDict[wid.description] = wid
        except:
            pass
    for k in valDict:
        try:
            valDict[k].value = defVal[k]
        except:
            pass

def processImgPath(imgPath, kwargs):
    image = mpimg.imread(imgPath)
    outP =(detectLanes(image, kwargs))
    displaySideBySide(image, outP)

def playWithParams(**kwargs):
    global PLAYED_PARAMS
    PLAYED_PARAMS = kwargs
    try:
        if kwargs["images"] == "all":
            for imgPath in IMAGES[2:]:
                processImgPath(imgPath, kwargs)
        else:
            processImgPath(kwargs["images"], kwargs)
            
    except:
        pass
        #print "EXCEPTION!", e
        #outP = image
    
w = interactive(playWithParams, **hyperParamsInteract)

import json
with open('params.json', 'r') as fp:
    global hyperParams
    try:
        hyperParams = json.load(fp)
    except:
        print("Failed loading params from params.json file!")
    
assignDefaults(w, hyperParams)
display(w)

In [7]:
import json

def saveParams():
    with open('params.json', 'w') as fp:
        json.dump(PLAYED_PARAMS, fp)
        
    print("Params saved to params.json!")
# Call this to save the params
#saveParams()